Este trabalho é referente à disciplina de Inteligência Artificial do curso de Bacharelado em Ciência da Computação na Universidade Federal do Pará. Ele propõe a implementação de uma versão do algoritmo **_Stochastic Hill Climbing_** (**_Simulated Annealing_**) e do **_Genetic Algorithm_** para a resolução do **problema das oito rainhas**.

Integrantes:

- [201804940002] Eduardo Gil Serrão Cardoso
- [201804940016] Gabriela Souza Maximino
- [201704940007] Igor Matheus Souza Moreira

As questões propostas pelo trabalho são citadas e respondidas abaixo.

# _Stochastic Hill Climbing: Simulated Annealing_

## Descreva a função objetivo utilizada na modelagem do problema.

A função objetivo utilizada verifica a existência de ataques de rainhas no tabuleiro na horizontal e na diagonal. Sua representação é mostrada abaixo:

`ha_ataque_horizontal = rainhas[indice_a] == rainhas[indice_b]`

`ha_ataque_diagonal = (indice_b - indice_a) == abs(rainhas[indice_b] - rainhas[indice_a])`

Dado que o tabuleiro é representado por um vetor com _n_ elementos, em que o valor de cada elemento da posição _i_ representa a linha onde a rainha se encontra nessa mesma coluna, percorre-se esse vetor varrendo para cada elemento da posição _indice_a_ os elementos seguintes que estão na posição _indice_b_. Caso algum tipo de ataque seja detectado, soma-se 1 ao valor do tabuleiro.

## Descreva a codificação utilizada para a solução candidata.

Para esse algoritmo, utilizou-se a codificação binária a fim de homogeneizar a codificação em relação à utilizada pelo Algoritmo Genético. Contudo, pode-se utilizar também a decimal. 

## Descreva a(s) heurística(s) e o(s) critério(s) de parada utilizados pelo algoritmo.

Foi utilizada a heurística de que as rainhas não podem estar em uma mesma coluna. 

No que diz respeito à condição de parada, adotaram-se dois critérios: 
1. a temperatura chegar a zero (i.e., percorrer todo o vetor de _arrays_ de temperaturas, cujo tamanho depende da temperatura inicial e variação); e
2. o valor mínimo da função objetivo (0) for alcançado.

## Execute 50 vezes o algoritmo e calcule: média e desvio padrão do número mínimo de iterações necessário para parar o algoritmo; média e desvio padrão do tempo de execução do algoritmo

## Construa dois gráficos:
- **Plotar a curva com número mínimo de iterações de cada execução.**
- **Plotar a curva com o tempo de execução do algoritmo de cada execução.**

## Mostre, pelo menos, duas soluções distintas encontradas pelo algoritmo.

## Comente e mostre o código fonte do algoritmo desenvolvido.

# _Genetic Algorithm_

## Descreva a função objetivo utilizada na modelagem do problema.

A função objetivo utilizada é a mesma do algoritmo anterior, uma vez que essa função foi definida na classe Tabuleiro, utilizada por ambos os algoritmos.

## Apresente a escolha e explique o funcionamento dos operadores que foram utilizados: seleção dos pais, cruzamento e mutação.

- Para a **seleção dos pais**, utilizou-se a técnica **_n-Way Tournament Selection_**, em que são selecionados _n_ indivíduos aleatórios da população para "lutar" entre si, ganhando o que tiver melhor _fitness_. Por padrão, _n_ é definido como 3. O torneio acontece duas vezes: uma para selecionar o primeiro pai e a segunda para selecionar o segundo.
- Para o **cruzamento**, utilizou-se a técnica de **cruzamento com ponto de corte aleatório**. Nessa técnica, uma posição aleatória _c_ do vetor é escolhida como ponto de corte. Quando a taxa de _crossover_ não é ultrapassada, corta-se o vetor do pai 1 da posição inicial até a posição _c_ e o vetor do pai 2 de _c_ até a última posição. Os cortes são concatenados gerando o filho 1, e o restante dos vetores "cortados" também são concatenados para gerar o filho 2. Caso a taxa de crossover seja ultrapassada, os pais tornam-se os filhos.
- Para a **mutação**, utilizou-se a técnica de **mutação com permutação aleatória**. Nessa técnica, duas posições do vetor são escolhidas aleatoriamente para serem permutadas, caso a taxa de mutação não seja ultrapassa. Caso seja, a mutação não ocorre.

## Execute 50 vezes o algoritmo e apresente, em forma de tabela, a melhor solução encontrada em cada execução, o valor da função objetivo desta solução encontrada, o tempo de execução e o número da geração em que o algoritmo parou.

## Calcule a média e o desvio padrão do valor da função objetivo do melhor indivíduo, do tempo de execução e o número da geração em que o algoritmo parou (três últimas colunas da tabela).

## Mostre, pelo menos, duas soluções distintas encontradas pelo algoritmo.

## Comente e mostre o código fonte do algoritmo desenvolvido.